In [ ]:
import os
import csv
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
#csv file path
path='/content/drive/My Drive/Colab Notebooks/security/data'.
fullpath=os.path.join(path, 'total.csv')
# 8:2=train:test
dataset = pd.read_csv(fullpath)
train_set, test_set = train_test_split(dataset, test_size = 0.2)

train_set.shape
test_set.shape



/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(50186, 80)

In [ ]:
print(test_set)

       Dst Port  Protocol  ...    Idle Min           Label
88174       443         6  ...  59800000.0   Infilteration
57542     53755         6  ...         0.0   Infilteration
223355       21         6  ...         0.0  FTP-BruteForce
7456       3389         6  ...         0.0          Benign
20169      3389        17  ...  14800000.0          Benign
...         ...       ...  ...         ...             ...
215418       21         6  ...         0.0  FTP-BruteForce
124887     8080         6  ...         0.0             Bot
36747     55550         6  ...         0.0          Benign
80717       443         6  ...   9332597.0   Infilteration
214027       21         6  ...         0.0  FTP-BruteForce

[50186 rows x 80 columns]


In [ ]:
print(test_set.iloc[0])

Dst Port                         443
Protocol                           6
Timestamp        01/03/2018 03:07:25
Flow Duration               60295141
Tot Fwd Pkts                       4
                        ...         
Idle Mean                   5.98e+07
Idle Std                           0
Idle Max                    5.98e+07
Idle Min                    5.98e+07
Label                  Infilteration
Name: 88174, Length: 80, dtype: object


In [ ]:
#print(dataset)
cols=["Flow Duration","Flow Pkts/s","Flow IAT Mean","Flow IAT Max","Flow IAT Min","Fwd Header Len","Bwd Header Len","Bwd Pkts/s","Init Fwd Win Byts"]
transactions=[]

for idx in range(len(train_set)):
  transaction=[]
  print(str(train_set.iloc[idx]['Dst Port']))
  transaction.append(str(train_set.iloc[idx]['Dst Port']))
  for col in cols:
    if train_set.iloc[idx][col]<100:
      transaction.append("L_"+col)
    elif train_set.iloc[idx][col]<1000:
      transaction.append("M_"+col)
    else:
      transaction.append("H_"+col)
  if train_set.iloc[idx]['Fwd Seg Size Min']<=20:
    transaction.append("L_FSSM")
  elif train_set.iloc[idx]['Fwd Seg Size Min']>20:
    transaction.append("H_FSSM")
  transaction.append(train_set.iloc[idx]['Label'])
  transactions.append(transaction)

In [ ]:
print(transactions[2])

['21', 'L_Flow Duration', 'H_Flow Pkts/s', 'L_Flow IAT Mean', 'L_Flow IAT Max', 'L_Flow IAT Min', 'L_Fwd Header Len', 'L_Bwd Header Len', 'H_Bwd Pkts/s', 'H_Init Fwd Win Byts', 'H_FSSM', 'FTP-BruteForce']


In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori


te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = apriori(df, min_support=0.35, use_colnames=True)

from mlxtend.frequent_patterns import association_rules
rules=association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3) 
rules.sort_values(by='lift',ascending=False).head()





,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1011,"(H_Flow IAT Mean, L_Bwd Header Len, H_Flow Dur...","(H_Flow IAT Max, L_FSSM, L_Fwd Header Len)",0.412146,0.386959,0.379338,0.920396,2.378535,0.219854,7.701172
965,"(H_Flow IAT Max, L_FSSM, L_Fwd Header Len)","(H_Flow IAT Mean, L_Bwd Header Len)",0.386959,0.412146,0.379338,0.980304,2.378535,0.219854,29.845696
972,"(H_Flow IAT Mean, L_Bwd Header Len)","(H_Flow IAT Max, L_FSSM, L_Fwd Header Len)",0.412146,0.386959,0.379338,0.920396,2.378535,0.219854,7.701172
1018,"(H_Flow IAT Max, L_FSSM, L_Fwd Header Len)","(H_Flow IAT Mean, L_Bwd Header Len, H_Flow Dur...",0.386959,0.412146,0.379338,0.980304,2.378535,0.219854,29.845696
1029,"(H_Flow IAT Mean, L_Bwd Header Len)","(H_Flow IAT Max, L_FSSM, L_Fwd Header Len, H_F...",0.412146,0.386959,0.379338,0.920396,2.378535,0.219854,7.701172
